![CoSAppLogo](../images/cosapp.svg)

<font color='orange'>**CoSApp**</font> tutorials on multimode systems

# Bouncing Ball

In [ ]:
from cosapp.base import System
import numpy as np


class PointDynamics(System):
    """Point mass dynamics"""
    def setup(self):
        self.add_inward("mass", 1.0)
        self.add_inward("acc_ext", np.zeros(3))
        self.add_inward("force_ext", np.zeros(3))

        self.add_outward("force", np.zeros(3))
        self.add_outward("acc", np.zeros(3))

    def compute(self):
        self.force = self.force_ext + self.mass * self.acc_ext
        self.acc = self.force / self.mass


class PointFriction(System):
    """Point mass ~ v^2 friction model"""
    def setup(self):
        self.add_inward('v', np.zeros(3), desc="Velocity")
        self.add_inward('cf', 0.1, desc="Friction coefficient")

        self.add_outward("force", np.zeros(3))

    def compute(self):
        self.force = (-self.cf * np.linalg.norm(self.v)) * self.v


class PointMassFreeFall(System):
    """Point mass free fall model with friction"""
    def setup(self):
        self.add_child(PointFriction('friction'), pulling=['cf', 'v'])
        self.add_child(PointDynamics('dynamics'), pulling={
            'mass': 'mass',
            'force': 'force',
            'acc_ext': 'g',
            'acc': 'a',
        })

        self.connect(self.friction, self.dynamics, {"force": "force_ext"})

        self.add_transient('v', der='a')
        self.add_transient('x', der='v')

        self.g = np.r_[0, 0, -9.81]
        self.exec_order = ['friction', 'dynamics']


class BouncingBall(PointMassFreeFall):
    """Extension of `PointMassFreeFall`, containing a rebound event
    and a rebound restitution coefficient.
    """
    def setup(self):
        super().setup()
        self.add_event('rebound', trigger="x[2] <= 0")
        self.add_inward('cr', 1.0, desc="Rebound restitution coefficient", limits=(0, 1))
        self.add_outward_modevar("n_rebounds", init=0, dtype=int)

    def transition(self):
        if self.rebound.present:
            self.n_rebounds += 1
            v = self.v
            if abs(v[2]) < 1e-6:
                v[2] = 0
            else:
                v[2] *= -self.cr


In [ ]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder

ball = BouncingBall('ball')
driver = ball.add_driver(RungeKutta(order=3, dt=0.01))
driver.time_interval = (0, 20)

# Add a recorder to capture time evolution in a dataframe
driver.add_recorder(
    DataFrameRecorder(includes=['x', 'v', 'a', 'norm(v)']),
    period=0.05,
)

# Define a simulation scenario
driver.set_scenario(
    init = {
        'x': np.zeros(3),
        'v': np.array([8, 0, 9.5]),
    },
    values = {
        'mass': 1.5,
        'cf': 0.20,
        'cr': 0.98,
    },
    stop = ball.rebound.filter("norm(v) < 1"),
)

ball.run_drivers()

# Retrieve recorded data
data = driver.recorder.export_data()
data = data.drop(['Section', 'Status', 'Error code'], axis=1)
time = np.asarray(data['time'])
traj = np.asarray(data['x'].tolist())


In [ ]:
# Plot results
import plotly.graph_objs as go

traces = [
    go.Scatter(
        x = traj[:, 0],
        y = traj[:, 2],
        mode = 'lines',
        name = 'numerical',
        line = dict(color='red'),
    )
]
layout = go.Layout(
    title = "Trajectory",
    xaxis = dict(title="x"),
    yaxis = dict(
        title = "z",
        scaleanchor = "x",
        scaleratio = 1,
    ),
    hovermode = "x",
)
go.Figure(data=traces, layout=layout)

In the next cell, we filter the recorded data in the vicinity of a rebound.
Notice that there are two entries at event time $t \approx 1.457$, showing the discontinuity in velocity (all other quantities are here continuous).

In [ ]:
data[data['time'].between(1.4, 1.6)]

Time driver property `recorded_events` contains a list of all recorded events.
Each element of the list is a named tuple `EventRecord(time, events)`, containing the occurrence time, and the list of cascading events at that time, starting by the primitive event.

In [ ]:
driver.recorded_events

Time driver property `event_data` contains the same fields as the recorder, but only at event times:

In [ ]:
driver.event_data.drop(['Section', 'Status', 'Error code'], axis=1)